In [1]:
import os
import json
import csv
import numpy as np
import pandas as pd
import re

In [83]:

path = '' ## path to tsv file with MQM annotations

df = pd.read_csv(path, sep='\t')
data=[]

with open(path) as f:
    for line in f.readlines():
        fields = line.strip().split('\t')
        if len(fields)>10:
            print('erroneous line:'+line)
            break 
        else:
            if len(fields)==9:
                fields.append('')
            data.append(fields)
    col = data[0]
    # read into df for visualisation purposes
    df =pd.DataFrame(data[1:], columns=col)
 
df.columns


Index(['system', 'doc', 'doc_id', 'seg_id', 'rater', 'source', 'target',
       'category', 'severity', ''],
      dtype='object')

In [84]:
import re 

def find_indices(astring, apattern):
    match=(re.search(apattern, astring))

    if match != None:
        return match.span()
    else:
        return None



In [85]:
   
pre_sid = ''
indices = {}
sources = {}
targets = {}
severities = {}

for i,r in df.iterrows():
    seg_id = str(r.seg_id).strip()
    doc_id = str(r.doc_id).strip()
    system = r.system.strip()
    doc = r.doc.strip()
    rater = r.rater.strip()
    source = r.source.strip()
    target = r.target.strip()
    severity = r.severity.strip().lower()
    main_source = source.replace("<v>","").replace("</v>","")
    main_source = re.sub(' +', ' ', main_source)
    main_target = target.replace("<v>","").replace("</v>","")
    main_target = re.sub(' +', ' ', main_target)
    
    sid = system + '_' + doc + '_' + doc_id + '_' + rater
    if pre_sid!=sid :
        pre_sid = sid
        indices[sid] = []
        targets[sid] = main_source
        sources[sid] = main_target 
    if severity == 'no-error':
        indices[sid].append([-1,-1,'no-error'])
    elif severity == 'minor':
        #find indices in target
        start_indices = find_indices(target,'<v>')
        end_indices = find_indices(target,'</v>')
        if start_indices != None and end_indices != None :
            if len(start_indices)>2 or len(end_indices)>2:
                print("LONGER than anticipated")
            else:
                start = start_indices[0]
                end = end_indices[0]-3
                span = main_target[start:end]
                if span.startswith(' '):
                    start = start+1
                if span.endswith(' '):
                    end = end-1
                arr = [start,end,'minor']
                indices[sid].append(arr)
        else:
            arr = [-1,-1,'no-error']
    else:
        start_indices = find_indices(target,'<v>')
        end_indices = find_indices(target,'</v>')
        if start_indices != None and end_indices != None :
            if len(start_indices)>2 or len(end_indices)>2:
                print("LONGER than anticipated")
            else:
                arr = [start_indices[0],end_indices[0],'major']
                indices[sid].append(arr)
        else:
            arr = [-1,-1,'no-error']
            
lines = []
for s in indices:
    spans = indices[s]
    source = sources[s]
    target = targets[s]
    if len(spans)>0:
        starts = []
        ends = []
        sevs = []
        for sp in spans:
            starts.append(str(sp[0]))
            ends.append(str(sp[1]))
            sevs.append(str(sp[2]))
        starts, ends, sevs = zip(*sorted(zip(starts, ends, sevs)))
        start = " ".join(starts)
        end = " ".join(ends)
        sev = " ".join(sevs)
        idx_splits = s.split('_')
        line = [idx_splits[0],idx_splits[1],idx_splits[2],idx_splits[3],source,target,start,end,sev]
        str_line = "\t".join(line)
        #print(str_line)
        lines.append(str_line)
    
    

In [87]:
outfile = '' ## path to output file for task 2
with open(outfile,'w') as wf:
    for line in lines:
        wf.write(line.strip()+'\n')
    

In [125]:
#2022

path = '/Users/chryssa/Documents/GitHub/wmt-qe-2023-data/task_2/MQM-data/2022/WMT2022_SharedTask_QualityEstimation_Task1_testset_zhcn-en - concatenated_errors_per_line.tsv'
#df = pd.read_csv(path, sep='\t')

data=[]

with open(path) as f:
    for line in f.readlines():
        fields = line.strip().split('\t')
        if len(fields)>35:
            print('erroneous line:'+line)
            break 
      
        data.append(fields)
    col = data[0]
    # read into df for visualisation purposes
    df =pd.DataFrame(data[1:], columns=col)
 

df.columns

Index(['_id', 'batch', 'typology', 'creation_date', 'translation_step',
       'content_type', 'customer_username', 'origin', 'source_language',
       'target_language', 'lp', 'task_annotation_id', 'annotator', 'fluency',
       'register', 'job_id', 'job_uid', 'idx', 'nugget_id', 'source', 'target',
       'mt', 'nr_words', 'returned_translations', 'sender', 'severity',
       'minor', 'major', 'critical', 'mqm', 'typology_error', 'error',
       'start_index', 'end_index'],
      dtype='object')

In [126]:
lines = []

for i,r in df.iterrows():
    rater = r.annotator
    source = r.source
    target = r.target
    seg_id = r.mt
    id = r._id
    
    mqm = float(r.mqm)
    if mqm < 100.0:
        sevs = " ".join(r.severity.split('/'))
        sevs = sevs.replace('critical','major')
        starts = " ".join(r.start_index.split('/'))
        ends = " ".join(r.end_index.split('/'))
        line = "\t".join(['MT_system1', id, seg_id, rater, source, target, starts, ends, sevs])
    else:
        line = "\t".join(['MT_system1', id, seg_id, rater, source, target, "-1", "-1", "no-error"])
    
    
    lines.append(line)
print(len(lines))
print(lines[0])

505
MT_system1	62d6d7e42ec5447f6d331f4d		septecho	让大学精神与时代精神同频共振-新华网	The University's Spirit and the Spirit of the Times - Official website	54 46	70 51	major major


In [127]:
outfile = '/Users/chryssa/Documents/GitHub/wmt-qe-2023-data/task_2/dev/2022_zh-en_test_processed.tsv'
with open(outfile,'w') as wf:
    for line in lines:
        wf.write(line.strip()+'\n')

In [16]:
path = '/Users/chryssa/Documents/GitHub/wmt-qe-2023-data/task_2/train/2021_zh-en_processed.tsv'
df = pd.read_csv(path,sep='\t', error_bad_lines=False, header=None)
df.columns = ['mt_system','doc_id','seg_id','annotator_id','target','source','start_ids','end_ids','errors']
df = df[['mt_system','doc_id','seg_id','annotator_id','source','target','start_ids','end_ids','errors']]

# for i,r in df.iterrows():
#     df.at[i,'seg_id'] = int(i)
    
# df['seg_id'] = df['seg_id'].astype(int)
df.to_csv(path,sep='\t',index=False)
df

/var/folders/my/8mkx3rq90gzg2pnm1txglr3m0000gn/T/ipykernel_41670/1192809191.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(path,sep='\t', error_bad_lines=False, header=None)


,mt_system,doc_id,seg_id,annotator_id,source,target,start_ids,end_ids,errors
0,hyp.Borderline,chinanews.com.229454,1,rater8,当地时间25日下午，一架小型飞机在德国北威州韦瑟尔县撞上一座多层住宅楼的顶层，事故共造成三人死亡。,"On the afternoon of the 25th local time, a sma...",130,139,major
1,hyp.DIDI-NLP,chinanews.com.229454,1,rater9,当地时间25日下午，一架小型飞机在德国北威州韦瑟尔县撞上一座多层住宅楼的顶层，事故共造成三人死亡。,"On the afternoon of the 25th local time, a sma...",0 130,5 139,major major
2,hyp.Facebook-AI,chinanews.com.229454,1,rater2,当地时间25日下午，一架小型飞机在德国北威州韦瑟尔县撞上一座多层住宅楼的顶层，事故共造成三人死亡。,"On the afternoon of the 25th local time, a sma...",138,141,minor
3,hyp.IIE-MT,chinanews.com.229454,1,rater3,当地时间25日下午，一架小型飞机在德国北威州韦瑟尔县撞上一座多层住宅楼的顶层，事故共造成三人死亡。,"On the afternoon of the 25th local time, a sma...",124 178 43,139 187 57,major major major
4,hyp.MiSS,chinanews.com.229454,1,rater5,当地时间25日下午，一架小型飞机在德国北威州韦瑟尔县撞上一座多层住宅楼的顶层，事故共造成三人死亡。,"On the afternoon of the 25th local time, a sma...",124,133,major
...,...,...,...,...,...,...,...,...,...
9475,hyp.metricsystem3,xinhua-zh-01.120586,650,rater8,饮食上可选择绿豆、薏仁等具有清热解暑、健脾开胃作用的食物，以及苦菜、苦瓜等苦味食物，可有助于...,"In the diet, you can choose foods such as mung...",191 54 62 91,211 63 67 167,major major minor major
9476,hyp.metricsystem4,xinhua-zh-01.120586,650,rater4,饮食上可选择绿豆、薏仁等具有清热解暑、健脾开胃作用的食物，以及苦菜、苦瓜等苦味食物，可有助于...,"Dietary choice of mung beans, barley and other...",160 63,200 87,minor minor
9477,hyp.metricsystem5,xinhua-zh-01.120586,650,rater1,饮食上可选择绿豆、薏仁等具有清热解暑、健脾开胃作用的食物，以及苦菜、苦瓜等苦味食物，可有助于...,Foods that have the effect of clearing heat an...,138 180,155 197,major minor
9478,ref.A,xinhua-zh-01.120586,650,rater1,饮食上可选择绿豆、薏仁等具有清热解暑、健脾开胃作用的食物，以及苦菜、苦瓜等苦味食物，可有助于...,"Therefore, mung beans and coix seeds should be...",148,162,major
